Implementation of TF-IDF algorithm (in chunks): 
1) create dictionary: with terms and with concepts names
2) create correlation matrix 
3) cut matrix according to length of terms list, to have only the part where we compare terms with concepts
4) find the most similar concept for every term
5) create df with term, concept, concept_id and how similar are they


In [2]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import spacy
nlp=spacy.load('en_core_web_lg')
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-05-19 17:45:55.047628: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-19 17:45:55.048698: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Data preporation

##### Snomed preprocessing

In [1]:
terms=pd.read_excel('concept_evaluation_new.xlsx')

NameError: name 'pd' is not defined

In [16]:
terms=terms['sourceName'].dropna(axis=0)

In [17]:
terms

1                                                 bleed
2                                             Bananas 1
3                                   triglyceride mg/ dl
4                                            heart rate
5                                             infection
                            ...                        
84    dessert e.g. ice cream mousse au chocolat pudd...
85                             TN pass chair stand test
86                               1 rest heart frequency
87                                 measurement left eye
88    malignant melanoma malignant melanoma acrolent...
Name: sourceName, Length: 83, dtype: object

In [6]:
concepts=pd.read_csv('CONCEPT.csv', on_bad_lines='skip', delimiter="\t", low_memory=False)

In [7]:
concepts=concepts[(concepts['vocabulary_id']=='SNOMED') & (concepts['standard_concept']=='S')]



In [8]:
concepts

,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason
205982,4001098,Radiating chest pain,Condition,SNOMED,Clinical Finding,S,10000006,20020131,20991231,NaN
205983,37392117,Urine tryptophan:creatinine ratio,Measurement,SNOMED,Observable Entity,S,1000001000000103,20160401,20991231,NaN
205985,46139744,Nu-Hope belt left small 6460C 70cm-78cm length...,Device,SNOMED,Physical Object,S,10000011000001109,20060501,20991231,NaN
205991,37398455,Urine threonine:creatinine ratio,Measurement,SNOMED,Observable Entity,S,1000011000000101,20160401,20991231,NaN
205992,46004596,Nu-Hope belt right large 6457A 90cm-100cm leng...,Device,SNOMED,Physical Object,S,10000111000001105,20060501,20991231,NaN
...,...,...,...,...,...,...,...,...,...,...
1240998,46004594,Nu-Hope belt right large 6457 90cm-100cm lengt...,Device,SNOMED,Physical Object,S,9999711000001105,20060501,20991231,NaN
1241001,37398453,Urine valine:creatinine ratio,Measurement,SNOMED,Observable Entity,S,999981000000107,20160401,20991231,NaN
1241002,46139743,Nu-Hope belt right large 6457 90cm-100cm lengt...,Device,SNOMED,Physical Object,S,9999811000001102,20060501,20991231,NaN
1241005,37398454,Urine tyrosine:creatinine ratio,Measurement,SNOMED,Observable Entity,S,999991000000109,20160401,20991231,NaN


##### HCHS preprocessing

In [18]:
terms=terms.str.lower().tolist()

In [19]:
#TODO add lemmatization to main code file creation 
ready=[]
for sentence in terms:
    token_list=[]
    doc=nlp(sentence)
    token_list=[token.lemma_ for token in doc]
    text = " ".join(token_list)
    ready.append(text)

In [20]:
terms=ready

##### Spacy preprocessing

In [24]:
nlp.Defaults.stop_words.remove('no')
nlp.Defaults.stop_words.remove('not')
nlp.Defaults.stop_words.remove('none')
nlp.Defaults.stop_words.remove('noone')
nlp.Defaults.stop_words.remove('back')

## TF-IDF part

In [26]:
def matrix_creation(data):
    tfidf=TfidfVectorizer()
    vecs=tfidf.fit_transform(data)
    corr_matrix = ((vecs * vecs.T).A)
    return corr_matrix

def similarity_matrix(matrix, dict):
    length=len(dict['terms'])
    return matrix[:length , length:]#the part where we compare terms with concepts

def similar_concept(matrix, dict):
    concepts_list=[]
    sim_list=[]
    concept_id_list=[]
    for i in range(matrix.shape[0]):
        concepts_list.append(dict['concepts'][np.argmax(matrix[i])])#TODO TOP3 
        sim_list.append(matrix[i].max())
        concept_id_list.append(np.argmax(matrix[i]))  
    return concepts_list, sim_list, concept_id_list

In [27]:
chunk_size=1000
df=pd.DataFrame()
df['Term']=terms
df['Concept']= np.nan
df['Concept_id']=np.nan
df['Similarity']=[0] * len(terms)

for chunk in np.array_split(concepts, chunk_size):
    chunk['concept_name']=chunk['concept_name'].replace({'X]': 'X] '}, regex=True).str.lower()#TODO put it to preprocessing part
    chunk = chunk[['concept_id', 'concept_name']]
    chunk_id=chunk['concept_id'].tolist()
    chunk_concept= list(map(str, chunk['concept_name']))
    ready=[]
    for sentence in chunk_concept:
        token_list=[]
        doc=nlp(sentence)
        token_list=[token.lemma_ for token in doc if not token.is_stop and not token.is_punct and token.text!='doctor']#TODO put 'doctor' into stop word list 
        text = " ".join(token_list)
        ready.append(text)
    chunk_concept=ready
    a={'terms': terms, 'concepts': chunk_concept} 
    document_list=a['terms'] + a['concepts']
    Correletaion_matrix=matrix_creation(document_list)
    Similarity_matrix=similarity_matrix(Correletaion_matrix, a)
    Concept, sim, concept_id= similar_concept(Similarity_matrix, a)
    p=[]
    for i in range(len(df['Similarity'])):
        if df['Similarity'][i]<sim[i]:
            df['Similarity'][i]=sim[i]
            df['Concept'][i]=Concept[i]
            df['Concept_id'][i]=chunk_id[concept_id[i]]
            
print(df)

                                                 Term  \
0                                               bleed   
1                                           bananas 1   
2                                 triglyceride mg/ dl   
3                                          heart rate   
4                                           infection   
..                                                ...   
78  dessert e.g. ice cream mousse au chocolat pudd...   
79                           tn pass chair stand test   
80                             1 rest heart frequency   
81                              measurement leave eye   
82  malignant melanoma malignant melanoma acrolent...   

                           Concept  Concept_id  Similarity  
0                            bleed    437312.0    1.000000  
1                              NaN         NaN    0.000000  
2                     triglyceride  45957223.0    0.553501  
3                       heart rate   4239408.0    1.000000  
4         

In [28]:
df.to_csv('result_1.csv')